In [1]:
import re
import numpy as np
import pandas as pd


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
RESULTS_FILE = "../gpt-prompting/results.xlsx"

2023-04-26 16:09:38.861942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 16:09:44.722478: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-26 16:09:44.723022: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-26 16:09:44.723058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
results_df = pd.read_excel(RESULTS_FILE)
results_df

,domain,prompt,output,hf_coherence
0,Sports strategy,write about a new attack formation in soccer t...,"The game of soccer has evolved over the years,...",17.542677
1,Sports strategy,write about a offensive strategy to counter ha...,"In tennis, hard shots are a common tactic used...",18.199308
2,Sports strategy,create a new strategy to shut out charging the...,"As an AI language model, I don't have personal...",16.015976
3,Sports strategy,create a novel approach to countering the quee...,"As an AI language model, I have knowledge on h...",5.796356
4,Operating systems,Describe the differences between the 6502 micr...,The 6502 microprocessor and the x86_64 instruc...,17.176355
...,...,...,...,...
174,Movies,Write an original movie plot that revolves aro...,Title: The Rubber Band Family\n\nThe Jones fam...,19.132065
175,Movies,Write an original movie plot that revolves aro...,Title: The Elastics Family\n\nThe Elastics are...,16.703779
176,Movies,"Write a movie review for the movie ""Garfield"".","As an AI language model, I don't have personal...",10.447511
177,Movies,"Write a movie review for the movie ""Garfield"".","As an AI language model, I do not have persona...",15.235296


In [3]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
# Remove new line characters
# data = [re.sub('\s+', ' ', sent) for sent in data]
data = results_df.output.apply(lambda sent: re.sub('\s+', ' ', sent))

# Remove distracting single quotes
data = data.apply(lambda sent: re.sub("\'", "", sent))

data = data.values.tolist()

<>:3: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
/tmp/ipykernel_410015/1943547870.py:3: DeprecationWarning: invalid escape sequence \s
  data = results_df.output.apply(lambda sent: re.sub('\s+', ' ', sent))


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['the', 'game', 'of', 'soccer', 'has', 'evolved', 'over', 'the', 'years', 'and', 'so', 'have', 'the', 'tactics', 'and', 'strategies', 'that', 'teams', 'use', 'to', 'outsmart', 'their', 'opponents', 'there', 'have', 'been', 'various', 'formations', 'used', 'in', 'soccer', 'each', 'with', 'its', 'own', 'strengths', 'and', 'weaknesses', 'however', 'in', 'recent', 'years', 'new', 'attack', 'formation', 'has', 'emerged', 'that', 'allows', 'teams', 'to', 'make', 'better', 'use', 'of', 'through', 'balls', 'on', 'offense', 'traditionally', 'teams', 'have', 'used', 'the', 'formation', 'which', 'involves', 'four', 'defenders', 'four', 'midfielders', 'and', 'two', 'strikers', 'this', 'formation', 'has', 'been', 'popular', 'because', 'it', 'provides', 'solidity', 'at', 'the', 'back', 'while', 'offering', 'enough', 'attacking', 'options', 'however', 'the', 'drawback', 'of', 'this', 'formation', 'is', 'that', 'it', 'can', 'be', 'predictable', 'and', 'lacks', 'creativity', 'in', 'the', 'final', 'thi

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'game', 'of', 'soccer', 'has', 'evolved', 'over', 'the', 'years', 'and', 'so', 'have', 'the', 'tactics', 'and', 'strategies', 'that', 'teams', 'use', 'to', 'outsmart', 'their', 'opponents', 'there', 'have', 'been', 'various', 'formations', 'used', 'in', 'soccer', 'each', 'with', 'its', 'own', 'strengths', 'and', 'weaknesses', 'however', 'in', 'recent', 'years', 'new', 'attack', 'formation', 'has', 'emerged', 'that', 'allows', 'teams', 'to', 'make', 'better', 'use', 'of', 'through_balls', 'on', 'offense', 'traditionally', 'teams', 'have', 'used', 'the', 'formation', 'which', 'involves', 'four', 'defenders', 'four', 'midfielders', 'and', 'two', 'strikers', 'this', 'formation', 'has', 'been', 'popular', 'because', 'it', 'provides', 'solidity', 'at', 'the', 'back', 'while', 'offering', 'enough', 'attacking', 'options', 'however', 'the', 'drawback', 'of', 'this', 'formation', 'is', 'that', 'it', 'can', 'be', 'predictable', 'and', 'lacks', 'creativity', 'in', 'the', 'final', 'third',

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['game', 'soccer', 'evolve', 'year', 'tactic', 'strategy', 'team', 'outsmart', 'opponent', 'various', 'formation', 'use', 'soccer', 'strength', 'weakness', 'however', 'recent', 'year', 'new', 'attack', 'formation', 'emerge', 'allow', 'team', 'make', 'well', 'ball', 'offense', 'traditionally', 'team', 'use', 'formation', 'involve', 'defender', 'midfielder', 'striker', 'formation', 'popular', 'provide', 'solidity', 'back', 'offer', 'enough', 'attack', 'option', 'however', 'drawback', 'formation', 'predictable', 'lack', 'creativity', 'final', 'third', 'new', 'formation', 'allow', 'ball', 'offense', 'formation', 'formation', 'involve', 'defender', 'central', 'defensive', 'midfielder', 'attack', 'midfielder', 'striker', 'key', 'difference', 'formation', 'traditional', 'central', 'defensive', 'midfielder', 'sit', 'front', 'defense', 'central', 'defensive', 'midfielder', 'provide', 'team', 'additional', 'defensive', 'cover', 'thus', 'allow', 'attack', 'midfielder', 'adventurous', 'forward', 

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 5), (4, 1), (5, 10), (6, 1), (7, 5), (8, 1), (9, 4), (10, 2), (11, 1), (12, 1), (13, 1), (14, 2), (15, 2), (16, 1), (17, 2), (18, 3), (19, 6), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 13), (30, 1), (31, 1), (32, 3), (33, 2), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 2), (40, 2), (41, 10), (42, 2), (43, 4), (44, 1), (45, 1), (46, 1), (47, 3), (48, 1), (49, 1), (50, 2), (51, 2), (52, 2), (53, 1), (54, 2), (55, 1), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 4), (67, 1), (68, 1), (69, 8), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 2), (78, 2)]]


In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
# Print the Keyword in the 10 topics
from pprint import pprint
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.025*"quarterback" + 0.020*"ai_language" + 0.019*"receiver" + '
  '0.019*"running_back" + 0.017*"however" + 0.015*"strategy" + 0.015*"model" + '
  '0.013*"watch" + 0.013*"quick" + 0.013*"counter"'),
 (1,
  '0.028*"player" + 0.026*"formation" + 0.026*"defense" + 0.023*"play" + '
  '0.023*"ball" + 0.022*"attack" + 0.021*"team" + 0.018*"create" + '
  '0.013*"make" + 0.013*"strategy"'),
 (2,
  '0.053*"oil" + 0.036*"surface" + 0.033*"pan" + 0.027*"layer" + '
  '0.027*"season" + 0.023*"process" + 0.020*"non_stick" + 0.018*"spouse" + '
  '0.017*"head" + 0.017*"create"'),
 (3,
  '0.045*"cook" + 0.041*"white_meat" + 0.038*"red_meat" + 0.030*"temperature" '
  '+ 0.025*"cooking" + 0.025*"time" + 0.023*"chicken" + 0.017*"flavor" + '
  '0.017*"method" + 0.015*"season"'),
 (4,
  '0.038*"movie" + 0.038*"film" + 0.029*"pointer" + 0.021*"ai_language" + '
  '0.020*"trigger" + 0.019*"model_dont" + 0.019*"however" + 0.018*"order" + '
  '0.018*"overall" + 0.018*"provide"'),
 (5,
  '0.056*"shot" + 

In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.689237290458064

Coherence Score:  0.5792251610369781


In [13]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/xtra/gable105/csci5541/env/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12     0.047706  0.000111       1        1  16.936949
1      0.298891  0.060184       2        1  16.232935
9     -0.086137  0.126458       3        1   7.110001
18    -0.023445  0.157072       4        1   6.927310
5      0.240219  0.054919       5        1   6.132443
3     -0.006238 -0.190119       6        1   5.034150
17    -0.021051  0.094989       7        1   5.008056
0      0.157746  0.003655       8        1   4.775666
11    -0.028378  0.019197       9        1   3.674260
15    -0.002954  0.010340      10        1   3.483107
13     0.031953 -0.207337      11        1   3.346942
2      0.009687 -0.198903      12        1   3.163611
10    -0.097890 -0.006799      13        1   2.905409
16    -0.122631  0.056782      14        1   2.663788
8     -0.095490  0.043235      15        1   2.343435
19    -0.065252 -0.110496      16        1   2.340705
14    -0.097581  0.034469      17        1   2.164140
4     -0.035859  0.022844      18        1   2.150746
6     -0.014511  0.037187      19        1   2.025808
7     -0.088784 -0.007789      20        1   1.580539, topic_info=             Term        Freq       Total Category  logprob  loglift
1136          tax  224.000000  224.000000  Default  30.0000  30.0000
255   instruction   73.000000   73.000000  Default  29.0000  29.0000
45       opponent   84.000000   84.000000  Default  28.0000  28.0000
1636    cent_coin   49.000000   49.000000  Default  27.0000  27.0000
143          shot   64.000000   64.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
962        return    3.569173   61.515785  Topic20  -4.3896   1.3004
1455         true    2.140684    9.358709  Topic20  -4.9008   2.6722
221         model    2.918068   42.018807  Topic20  -4.5910   1.4802
467   information    2.412644   21.613987  Topic20  -4.7812   1.9548
116          many    2.194547   19.004369  Topic20  -4.8760   1.9887

[1032 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
497       2  0.248398  ability
497       4  0.372597  ability
497       7  0.248398  ability
497       8  0.082799  ability
497      10  0.041400  ability
...     ...       ...      ...
78        8  0.064642     year
78        9  0.077570     year
78       10  0.012928     year
78       17  0.051714     year
203       2  0.941023     zone

[1870 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 2, 10, 19, 6, 4, 18, 1, 12, 16, 14, 3, 11, 17, 9, 20, 15, 5, 7, 8])